<a href="https://colab.research.google.com/github/HuangJin-De/am_NH/blob/main/discussion/20231115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! rm -rf am_NH
! git clone https://github.com/HuangJin-De/am_NH.git

Cloning into 'am_NH'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 175 (delta 70), reused 113 (delta 35), pack-reused 0
Receiving objects: 100% (175/175), 6.15 MiB | 19.50 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [6]:
import numpy as np
import matplotlib.pyplot as plt

path='am_NH'
filen=path+'/train_data/zm_10days_ori_data.dat'

data=np.fromfile(filen,dtype=np.float32)
data=data.reshape(-1,42)

y1=np.copy(data[:,0])
print(y1[0:10])
y2=np.copy(data[:,1])
print(y2[0:10])
x=np.copy(data[:,2:43])
print(x[0:10,0])

print(y1.shape,y2.shape,x.shape)


[-0.00886129 -0.10451781 -0.12247203 -0.07673766 -0.06012047  0.01127541
 -0.01339631 -0.12135324  0.02029838  0.1327517 ]
[ 0.05676877 -0.0081835  -0.05779969  0.04061143  0.0752062  -0.02967101
 -0.10725513 -0.0139739  -0.04375202 -0.0499122 ]
[27.807383 29.223433 30.451048 34.08071  28.950182 24.049631 23.813913
 25.54383  22.154768 16.67688 ]
(4662,) (4662,) (4662, 40)


estimations of b11, b12, b21, and b22.